In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_train = pd.read_csv('df_train.csv', sep=';')
df_test = pd.read_csv('df_test.csv', sep=';')

In [ ]:
df_train['Data'] = df_train.Data.apply(lambda s: list(map(int, s.split(','))))
df_train['Target'] = df_train.Target.apply(lambda s: list(map(int, s.split(','))))
df_test['Data'] = df_test.Data.apply(lambda s: list(map(int, s.split(','))))

In [ ]:
m = []
for x in df_train['Data']:
  for y in x:
    m.append(y)
m = list(set(m))
m.sort()

In [ ]:
import torch

In [ ]:
m_0 = []
for x in range(len(m)):
  m_0.append(0)
def i_hate_x(frame):
  data_transformed = []
  for x in frame:
    m_1 = m_0.copy()
    for y in x:
      m_1[m.index(y)]+=1
    data_transformed.append(m_1)
  data_T = []
  for i in range(len(m)):
    to_transp = []
    for j in range(len(data_transformed)):
      to_transp.append(data_transformed[j][i])
    data_T.append(to_transp)
    to_pd = {}
  for x in range(len(m)):
    to_pd[m[x]] = data_T[x]
  df = pd.DataFrame(to_pd)
  return(df)




def i_hate_y(frame):
  data_transformed = []
  for x in frame:
    m_1 = m_0.copy()
    for y in x:
      m_1[m.index(y)]=1
    data_transformed.append(m_1)
  data_T = []
  for i in range(len(m)):
    to_transp = []
    for j in range(len(data_transformed)):
      to_transp.append(data_transformed[j][i])
    data_T.append(torch.tensor(to_transp, dtype=torch.long))
  return(data_T)


In [ ]:
X_tr = torch.tensor(i_hate_x(df_train['Data']).values, dtype=torch.float32)
Y_tr = i_hate_y(df_train['Target'])

In [ ]:
import numpy as np

In [ ]:
X_ts = torch.tensor(i_hate_x(df_test['Data']).values, dtype=torch.float32)

In [ ]:
dt = []
for i in range(len(Y_tr[0])):
  dt.append([])

In [ ]:
data = []
def ans(df):
  for x in df:
    ebat = {}
    for y in x:
      if y in ebat.keys():
        ebat[y]+=1
      else:
        ebat[y] = 1
    ebat = dict(sorted(ebat.items(), key=lambda item: item[1], reverse=True))
    ebat_k = list(ebat.keys())
    ans = []
    for i in range(min(10, len(ebat_k))):
      ans.append(ebat_k[i])
    data.append(ans)

In [ ]:
data = []
ans(df_test['Data'])

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
for xyz in range(len(Y_tr)):
  pizda = torch.nn.Sigmoid()
  print(xyz)
  model = torch.nn.Sequential(
    torch.nn.Linear(184,184),
    torch.nn.ELU(),
    torch.nn.Linear(184,184),
    torch.nn.ELU(),
    torch.nn.Linear(184,1)
  )
  loss_function = torch.nn.BCEWithLogitsLoss()
  #optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

  train_dataset = TensorDataset(X_tr, Y_tr[xyz])
  train_dataloader = DataLoader(train_dataset, batch_size=128)
  val_dataset = TensorDataset(X_ts, Y_tr[xyz])
  val_dataloader = DataLoader(val_dataset, batch_size=128)

  criterion = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters())

  loaders = {"train": train_dataloader}

  max_epochs = 10
  accuracy = {"train": [], "valid": []}
  for epoch in range(max_epochs):
      for k, dataloader in loaders.items():
          epoch_correct = 0
          epoch_all = 0
          for x_batch, y_batch in dataloader:
              if k == "train":
                  model.train()
                  optimizer.zero_grad()
                  outp = model(x_batch)
                  probabilities = pizda(outp.detach())
                  preds = (probabilities > 0.5).type(torch.long)
              correct =  torch.tensor(sum(i[0] == i[1] for i in zip(preds, y_batch)))
              if k == "train":
                loss = loss_function(outp.flatten().float(), y_batch.flatten().float())
                loss.backward()
                optimizer.step()

  model.eval()
  predictions = np.array([])
  for x_batch, _ in val_dataloader:
      outp =  model(x_batch)
      probabilities = pizda(outp.detach())# YOUR CODE. Compute probabilities
      preds = (probabilities > 0.5).type(torch.long)
      predictions = np.hstack((predictions, preds.numpy().flatten()))


  for i in range(len(predictions)):
    if predictions[i]:
      if (not m[xyz] in data[i]) and (len(data[i])<10):
        data[i].append(m[xyz])


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183


In [ ]:
print(data)

[[6011, 4814, 6010, 4829, 5499, 5541, 5411, 5912, 5399], [6011, 6010, 5411, 4814, 5533, 5977, 5661, 5691, 5983, 5331], [6011, 6010, 5499, 4814, 5411, 5814, 4829, 5812, 5311, 5200], [6011, 4814, 5964, 5411, 4829, 5912, 6010, 6012, 5814, 5309], [4814, 6011, 6010], [6011, 4814, 5691, 5999, 5977, 5921, 5411, 5992, 5541, 6010], [4814, 6011, 5499, 4829, 5999, 5411, 5921, 5812, 5912, 5331], [5411, 4814, 5499, 6011, 4829, 5541, 5964, 5977, 5300, 5814], [5411, 4814, 5541, 6011, 6010, 5499, 5311, 5814, 5983, 4812], [6011, 4829, 4814, 6010, 5411, 5541, 5945, 5912, 5814, 5944], [6011, 4829, 5533, 4814, 6010, 5712, 5499, 6051, 6536, 5722], [6011, 4829, 4814, 5331, 6010, 5411, 5499, 5999, 5541, 5200], [5411, 6011, 5814, 4829, 4814, 5541, 5983, 4900, 9399, 4112], [7399, 6011, 4814, 5499, 4829, 5411, 6010, 5912, 5331, 5977], [6011, 4814, 5331, 5411, 5541, 4829, 5912, 5499, 6010], [5331, 6011, 5499, 5411, 5983, 5812, 4829, 5811, 5921, 5533], [6011, 6010, 4814, 5411, 4829, 5541, 5983, 4214, 5814, 7832],

In [ ]:
df_test['Predicted'] = data
submission_baseline_2 = df_test[['Id', 'Predicted']]
submission_baseline_2['Predicted'] = submission_baseline_2.Predicted.astype(str).str.replace(',', '')
submission_baseline_2.to_csv('submission_prob_lr.csv', index=False)

In [ ]:
#csr_data = csr_matrix(X_tr.values)

In [ ]:
#knn = NearestNeighbors(n_jobs = -1)

#knn.fit(csr_data)

NearestNeighbors(n_jobs=-1)

In [ ]:
#to_ans = i_hate_it(df_test['Data'])
#csr_data_test = csr_matrix(to_ans.values)

In [ ]:
'''import torch
x_tr = torch.tensor(X_tr.values)
x_tr.shape
ta = torch.tensor(to_ans.values)

In [ ]:
'''top10_codes = df_train['Data'].explode().value_counts().head(10)

In [ ]:

'''def pred():
  for x in range(len(data)):
    if len(data[x])<10:
      distances, indices = knn.kneighbors(csr_data_test[x], n_neighbors = 10)
      distances = distances.flatten()
      indices = indices.flatten()
      p = {}
      for x in range(184):
        for y in range(len(indices)):
          if x_tr[indices[y]][x]:
            if not m[x] in p.keys():
              p[m[x]] = distances[y]*x_tr[indices[y]][x]*10
            else:
              p[m[x]] += distances[y]*x_tr[indices[y]][x]*10
      p = dict(sorted(p.items(), key=lambda item: item[1], reverse=True))
      p_k = list(p.keys())
      i = 0
      while len(data[x])<10:
        if not p_k[i] in data:
            data[x].append(p_k[i])
        i+=1



In [ ]:
'''data = []
ans(df_test['Data'])
#pred()

df_test['Predicted'] = data


In [ ]:
'''df_test.head()

,Id,Data,Predicted
0,0,"[4814, 4814, 6011, 6011, 6010, 6011, 6011, 481...","[6011, 4814, 6010, 4829, 5499, 5541, 5411, 591..."
1,1,"[6010, 6011, 6010, 5411, 5411, 5977, 6011, 601...","[6011, 6010, 5411, 4814, 5533, 5977, 5661, 569..."
2,2,"[4814, 6011, 5251, 6011, 7832, 5641, 5814, 482...","[6011, 6010, 5499, 4814, 5411, 5814, 4829, 581..."
3,3,"[6011, 4722, 4722, 4722, 4814, 6011, 6011, 482...","[6011, 4814, 5964, 5411, 4829, 5912, 6010, 601..."
4,4,"[4814, 4814, 4814, 6011, 4814, 4814, 4814, 481...","[4814, 6011, 6010]"
